In [ ]:
## imports
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pycbem.constants import (eps_0, mu_0)
from pycbem.field import (efield, hfield)
from pycbem.utils.dataloader import (load_tissue_diel_properties,
                                     load_antenna_el_properties)
from pycbem.utils.integrate import elementwise_dblquad
from pycbem.utils.viz import fig_config

In [ ]:
## constants
# dry skin bhte parameters from IT'IS database
# https://itis.swiss/virtual-population/tissue-properties/database/
k = 0.37       # thermal conductivity [W/m/°C]
rho = 1109.    # dry skin density [kg/m^3]
C = 3391.      # specific heat of dry skin [Ws/kg/°C]
m_b = 1.76e-6  # blood perfusion [m^3/kg/s], equivalent to 106 mL/min/kg

# for blood
k_b = 0.52     # thermal conductivity [W/m/°C]
rho_b = 1000.  # kg/m^3
C_b = 3617.    # specific heat of blood [J/kg/°C]

# additional parameters
h_0 = 10.      # heat convection coefficient skin surface - air [W/m^2/°C]
T_a = 37.      # arterial temperature [°C]     
T_c = 37.      # body core temperature [°C]
T_f = 25.      # surrounding air temperature [°C]
Q_m = 33800.   # metabolic heat generation [W/m^3]

In [ ]:
# frequency
f = 10e9

# conductivity, relative permitivity, tangent loss and penetration depth (Gabriel et al. 1996)
sigma, eps_r, tan_loss, pen_depth = load_tissue_diel_properties('skin_dry', f)

# `pen_depth` is the energy penetration depth into tissue, which is defined as the distance 
# beneath the surface at which the SAR has fallen to a factor of 1/e below that at the surface;
# one-half of the more commonly reported wave penetration depth
pen_depth = pen_depth / 2

# air (vacuum) resistance 
Z_air = np.sqrt(mu_0 / eps_0)

# dry skin resistance
Z_skin_dry = np.sqrt(mu_0 / (eps_r * eps_0))

# energy (Fresnel) transmission coefficient into the tissue
T_tr = 2 * Z_skin_dry / (Z_air + Z_skin_dry)

# effective radiated skin area
target_area = (0.02, 0.02)  # 2 x 2 cm2
A = target_area[0] * target_area[1]
h = - 15 / 1000  # distance from the antenna

# human head
head_coords = pd.read_csv(os.path.join('data', 'head_ijnme_simpl.csv'),
                          names=['y', 'x', 'z'])
head_coords = head_coords + 0.05
target_area_coords = head_coords[
    (head_coords['y'] < 0) &
    (head_coords['x'] > 0) & (head_coords['x'] < target_area[0]) &
    (head_coords['z'] > 0) & (head_coords['z'] < target_area[1])]
target_area_coords.reset_index(drop=True, inplace=True)

# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j

# antenna position -- coordinates
xs = antenna_data.x.to_numpy() 
xs = xs + (target_area[0] - xs[-1]) / 2.
ys = np.zeros_like(xs) + target_area_coords['y'].min() + h
zs = np.zeros_like(xs)
zs = zs + (target_area[1] - zs[-1]) / 2.

In [ ]:
# efield
E = target_area_coords.apply(
    lambda row : efield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
E.columns = ['Ex', 'Ey', 'Ez']
E_abs = E.apply(
    lambda row : np.sqrt(row['Ex'] ** 2 + row['Ey'] ** 2 + row['Ez'] ** 2),
    axis=1)
E.loc[:, 'E_abs'] = E_abs

# hfield
H = target_area_coords.apply(
    lambda row: hfield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
H.columns = ['Hx', 'Hy', 'Hz']
H_abs = H.apply(
    lambda row: np.sqrt(row['Hx'] ** 2 + row['Hy'] ** 2 + row['Hz'] ** 2),
    axis=1)
H.loc[:, 'H_abs'] = H_abs

# update dataframe
target_area_calc = pd.concat([target_area_coords, E, H], axis=1)

# power density
def power_density_magnitude(row):
    return (np.sqrt(np.power(row.Ey * row.Hz.conjugate() - row.Ez * row.Hy.conjugate(), 2)
            + np.power(row.Ex * row.Hz.conjugate(), 2)
            + np.power(row.Ex * row.Hy.conjugate(), 2)))
I0m = target_area_calc.apply(power_density_magnitude, axis=1, result_type='expand')
I0m.columns = ['I0m']

# update dataframe
target_area_calc.loc[:, 'I0m'] = I0m

In [ ]:
fig_config(nrows=1, ncols=2, scaler=2)
fig, ax = plt.subplots(1, 2, sharey=True, squeeze=True)
ax[0].scatter(head_coords['x'], head_coords['z'], color='k', alpha=0.03)
im = ax[0].scatter(target_area_calc['x'], target_area_calc['z'], c=np.abs(target_area_calc['I0m']))
ax[0].plot(xs, zs, 'r-', linewidth=2, label='1/2 dipole (xz-plane)')
ax[0].set_xlabel('$x$ [m]')
ax[0].set_ylabel('$z$ [m]')
ax[1].scatter(head_coords['y'], head_coords['z'], color='k', alpha=0.03)
im = ax[1].scatter(target_area_calc['y'], target_area_calc['z'], c=np.abs(target_area_calc['I0m']))
ax[1].plot(ys[0], zs[0], 'ro', label='1/2 wave dipole (yz-plane)')
ax[1].set_xlabel('$y$ [m]')

fig.colorbar(im, ax=ax, label=r'I0 [W/m$^2$]')
fig.legend(bbox_to_anchor=(0.51, 1.01), ncol=1, frameon=True, edgecolor='k')

plt.show()